In [2]:
%matplotlib inline

import math
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#### API reference
  - https://www.tensorflow.org/api_docs/python/tf/contrib/layers  

### Convolutional layer
  - 이미지가 인풋인 경우 tf.contrib.layers.conv2d()로 간단히 적용 가능
    - 인풋: [batch_size, height, width, channel] 크기의 4-dim 데이터
    - 주요 함수 파라미터
      - inputs: 인풋 데이터
      - num_outputs: output dimension
      - kernel_size: 필터 크기
      - stride: stride 크기 (몇 칸씩 움직일 것인가) (default: 1)
      - padding: 양 옆 (가장자리)에 몇 개의 0을 붙일지에 대한 방법 
        - SAME: 입력/출력 데이터 크기 유지되도록 padding
        - VALID: padding 없이 진행
    - 추가적으로 제공되는 기능 함수 파라미터
      - activation_fc: convolution 연산 후 적용 할 activation 함수 (default: relu)
      - normalizer_fn: dropout, batch-normalization과 같은 함수 (default: None)
      - weights_initializer: weight 초기화 함수 (default: xavier_initializer)
      - weights_regularizer: L2 normalization과 같은 함수 (default: None)
      - biases_initializer
      - biases_initializer
      - <img src="../resource/convolution.png" width="600" height="400">

#### convolutional layer 예제 1

In [8]:
# 인풋 데이터를 위한 placeholder 정의
x = tf.placeholder(tf.float32, shape=(1,6,6,1))

# 1. num_output (1), 필터 크기 (3,3), stride (2)의 convolutional layer 
conv1 = tf.contrib.layers.conv2d(x, 1, [3,3], 2) # relu 자동 적용
"""
    위의 conv2d 함수 호출은 사실상 아래의 식과 같음
    conv1 = tf.contrib.layers.conv2d(x, 1, [3,3], 2, activation_fn=None)
    conv1 = tf.nn.relu(conv1)
"""

# 2. relu 비 적용하기
conv_noReLu = tf.contrib.layers.conv2d(x, 1, [3,3], 2, activation_fn=None)

# 3. dropout 적용
conv_dropout = tf.contrib.layers.conv2d(x, 1, [3,3], 2,\
                   normalizer_fn=tf.nn.dropout, \
                   normalizer_params={'keep_prob':0.8})
"""
    위의 conv2d 함수 호출은 사실상 아래의 식과 같음
    conv_dropout = tf.contrib.layers.conv2d(x, 1, [3,3], 2, activation_fn=None)
    conv_dropout = tf.nn.relu(conv_dropout)
    conv_dropout = tf.nn.dropout(conv_dropout)
"""

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    inp = np.random.randn(1,6,6,1)
    print 'input data:\n', inp[0,:,:,0], '\n'
    
    conv1_res = sess.run(conv1, {x:inp})
    print 'conv1 filer shape: ', conv1_res.shape
    print 'conv1:\n', conv1_res[0,:,:,0], '\n'
    
    conv_noReLu_res = sess.run(conv_noReLu, {x:inp})
    print 'conv_noReLu filer shape: ', conv_noReLu_res.shape
    print 'conv_noReLu:\n', conv_noReLu_res[0,:,:,0], '\n'
    
    conv_dropout_res = sess.run(conv_dropout, {x:inp})
    print 'conv_dropout filer shape: ', conv_dropout_res.shape
    print 'conv_dropout:\n', conv_dropout_res[0,:,:,0], '\n'

input data:
[[ 0.72779825  0.27322427  0.88988946 -1.04589035  1.08557927  0.56564219]
 [-0.61263425 -0.2192001   0.74078697 -0.68975175  0.83527752  0.82788006]
 [-0.14529889  1.45911961  1.82246006  0.98625156  2.00077984 -1.90555897]
 [ 0.90973621  0.643344   -0.43486309 -0.82414908 -1.62529854 -0.96242717]
 [-1.74158796  0.20903698 -0.73074806 -0.01209285  0.0477045   0.75997254]
 [-1.42037317 -0.48331209 -0.29589238 -2.5517964   0.22451886  0.42365891]] 

conv1 shape:  (1, 3, 3, 1)
conv1:
[[ 0.15481517  1.76303911  0.1844805 ]
 [ 0.          0.00429359  0.18552452]
 [ 0.95634812  1.49579477  0.        ]] 

conv_noReLu shape:  (1, 3, 3, 1)
conv_noReLu:
[[ 0.01683076 -1.00343883 -1.48803651]
 [ 0.29309848 -1.58401656 -1.40480816]
 [ 0.64232981  0.4992874   0.16121766]] 

conv_dropout shape:  (1, 3, 3, 1)
conv_dropout:
[[ 0.30555835  0.          0.        ]
 [ 2.02985358  1.89041007  0.        ]
 [ 0.          0.          0.21663754]] 



#### convolutional layer 예제 2

In [20]:
# 인풋 데이터를 위한 placeholder 정의
x = tf.placeholder(tf.float32, shape=(1,10,10,1))

# num_output (1), 필터 크기 (5,5), stride (1)의 convolutional layer
# 1. SAME으로 padding 주기
same_conv = tf.contrib.layers.conv2d(x, 1, [5,5], 1, padding='SAME')

# 2. VALID로 padding 주기
valid_conv = tf.contrib.layers.conv2d(x, 1, [5,5], 1, padding='VALID')

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    inp = np.random.randn(1,10,10,1)
    print 'shape of input data:', inp.shape
    
    same_res = sess.run(same_conv, {x:inp})
    print "shape of 'same' padding conv result: ", same_res.shape
    
    valid_res = sess.run(valid_conv, {x:inp})
    print "shape of 'valid' padding conv result: ", valid_res.shape

shape of input data: (1, 10, 10, 1)
shape of 'same' padding conv result:  (1, 10, 10, 1)
shape of 'valid' padding conv result:  (1, 6, 6, 1)


### Max pooling layer
  - tf.contrib.layers.max_pool2d()로 간단히 적용 가능
  - 인풋: [batch_size, height, width, channel] 크기의 4-dim 데이터
  - 함수 파라미터
    - inputs: 인풋 데이터
    - num_outputs: output dimension
    - kernel_size: 필터 크기
    - stride: stride 크기 (몇 칸씩 움직일 것인가) (default: 2)
    - padding: 양 옆 (가장자리)에 몇 개의 0을 붙일지에 대한 방법 
      - SAME: 입력/출력 데이터 크기 유지되도록 padding
      - VALID: padding 없이 진행
      - <img src="../resource/pooling.png" width="300" height="300">

#### pooling layer 예제

In [15]:
# 인풋 데이터를 위한 placeholder 정의
x = tf.placeholder(tf.float32, shape=(1,4,4,1))

# 1.  필터 크기 (2,2), stride (2)의 convolutional layer 
pooling = tf.contrib.layers.max_pool2d(x, [2,2], 2)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    inp = np.random.randn(1,4,4,1)
    print 'input data:\n', inp[0,:,:,0], '\n'
    
    pooling_res = sess.run(pooling, {x:inp})
    print 'pooling filer shape: ', pooling_res.shape
    print 'pooling:\n', pooling_res[0,:,:,0], '\n'

 input data:
[[-1.4907695   0.77464655  0.26188201 -1.12387434]
 [-1.10306545 -2.36949659 -0.7333842  -1.9290388 ]
 [ 0.986132    0.71187773  0.39659664  1.47691859]
 [-0.93051237 -0.19330032 -0.32219061  1.21276647]] 

pooling shape:  (1, 2, 2, 1)
pooling:
[[ 0.77464652  0.26188201]
 [ 0.98613203  1.47691858]] 

